In [1]:
import PyDSTool
from PyDSTool.Toolbox import phaseplane as pp
import matplotlib
from pylab import *
import copy
import math
matplotlib.rcParams.update({'font.size': 30})


In [2]:
def plotContPartial(DSarg,fixed_points,x,y,title,maxnum,step,col='k'):
        DSargs = copy.deepcopy(DSarg)
        freepar=x
        DSargs.pdomain={x:[0,100000]}
        odex = PyDSTool.Generator.Vode_ODEsystem(DSargs)
        odex.set(ics=fixed_points[0])
        PCargs = PyDSTool.args(name='EMTest',type='EP-C')
        PCargs.freepars =[x]
        PCargs.MaxNumPoints = maxnum ## max number for each call of forward and backward
        PCargs.MaxStepSize = 1e+2
        PCargs.MinStepSize = 1e-2
        PCargs.StepSize = step
        PCargs.StopAtPoints = ['B'] ##  stops searching once this point is hit
        PCargs.LocBifPoints = 'all' ##
        PCargs.SaveEigen = True

        PyCont = PyDSTool.ContClass(odex) # Set up continuation class
        PyCont.newCurve(PCargs)
        PyCont['EMTest'].forward()
        PyCont['EMTest'].backward()
        PyCont.display([x,y],stability=True,color=col)
        PyCont.plot.toggleLabels("off")
        plt.title('')
        plt.xlabel(x+" (molecules)",fontsize=20)
        plt.ylabel(y+" (molecules)",fontsize=20)
        #plt.savefig(title+".png",bbox_inches='tight')
        #plt.show()
        #plt.close()



In [3]:
def reduce_fp(fixed_points):

        final = []
        for el in fixed_points:
                if len(final)>0:
                        addI=True
                        for item in final:
                                if np.abs(el['u']-item['u'])<0.001:
                                        if np.abs(el['mz']-item['mz'])<0.001:
                                                addI=False
                        if addI:
                                final+=[el]

                else:
                        final+=[el]

        return tuple(final)



In [4]:
DSargs = PyDSTool.args(name='emt',checklevel=2)
DSargs.fnspecs = {'H':(['X','X0','nX','lamdaX'],'lamdaX+(1.-lamdaX)/(1.+(X/X0)**nX)')}

DSargs.pars = { 'ga':10.,'ka':0.1,'gb':10.,'kb':0.1,
                'a0a':2,'naa':2,'laa':5,'b0a':3,'nba':2,'lba':8,
                'a0b':2,'nab':2,'lab':5,'b0b':3,'nbb':2,'lbb':8}


DSargs.varspecs = { 'a':'ga*H(a,a0a,naa,laa)*H(b,b0a,nba,lba)-ka*a','b':'gb*H(a,a0b,nab,lab)*H(b,b0b,nbb,lbb)-kb*b'}

In [8]:
DSargs.ics = {'a':5,'b':0}
DSargs.xdomain = {'a':[0,10000],'b':[0,10000]}
DSargs.tdomain = [0,5000]
DSargs.algparams = {'init_step':0.1}

ode = PyDSTool.Generator.Vode_ODEsystem(DSargs)
Ffixed_points = pp.find_fixedpoints(ode,n=6,maxsearch=1e+6,eps=1e-10)

fixed_points = reduce_fp(Ffixed_points)
print("Reduced")
print(fixed_points)


Reduced
({'a': 3999.9972312478676, 'b': 3999.9972312478676},)
